In [1]:
!pip install pymssql


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

In [3]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [4]:

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data['rnk'] = data.ret.rank(pct=True)

In [7]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)

model = RandomForestRegressor(
   random_state=0
)

pipe = make_pipeline(
  transform1, 
  poly,
  transform2,
  model
)

In [8]:
param_grid = {
    "randomforestregressor__max_depth": [4, 6, 8]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m"]]
y = data["rnk"]

cv.fit(X, y)

dump(cv, "forest2.joblib")

['forest2.joblib']